In [3]:
import warnings
warnings.filterwarnings('ignore')

import torch

from datasets import load_dataset, load_dataset_builder

## 1. Introduction

We will be using https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english for sentiment analysis.

This model was trained on SST-2 dataset which is a dataset for binary sentiment classification. It is composed of sentences extracted from movie reviews and annotated with a sentiment label. The task is to predict the sentiment of a given sentence.

In [4]:
ds_builder = load_dataset_builder('stanfordnlp/sst2')
ds_builder.info.features

{'idx': Value(dtype='int32', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [5]:
initial_dataset = load_dataset('stanfordnlp/sst2')
initial_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [6]:
initial_dataset['train'][0]

{'idx': 0,
 'sentence': 'hide new secretions from the parental units ',
 'label': 0}

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(model)

In [8]:
tokenizer(initial_dataset['train'][0]['sentence'])

{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
def encode(example):
    return tokenizer(example['sentence'], truncation=True, padding="max_length")

tokenized_dataset = initial_dataset['train'].map(encode, batched=True)
tokenized_dataset

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 67349
})

In [10]:
tokenizer.decode(tokenized_dataset[0]['input_ids'])

'[CLS] hide new secretions from the parental units [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [11]:
tokenized_dataset.set_format(type='torch', columns=['sentence', 'input_ids'])
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=32)

## 2. Create mapping dataset for first-layer network encryption

The idea here is to first tokenize and then encrypt the tokens, so that we have a mapping of plain tokens to encrypted tokens. This mapping will be used to train the first-layer network.

In [ ]:
# Simple Cesear cipher encryption just for testing purposes
def encrypt_tokens(example, shift=3):
    encrypted_input_ids = [(token_id + shift) % tokenizer.vocab_size for token_id in example['input_ids']]
    example['encrypted_input_ids'] = encrypted_input_ids
    return example


In [13]:
encrypted_dataset = tokenized_dataset.map(encrypt_tokens, batched=True)

Map: 100%|██████████| 67349/67349 [00:01<00:00, 40081.79 examples/s]


In [14]:
encrypted_dataset.set_format(type='torch', columns=['encrypted_input_ids', 'input_ids'])

In [26]:
train_dataset = encrypted_dataset.shuffle(seed=42).select(range(1000))
val_dataset = encrypted_dataset.shuffle(seed=42).select(range(200)) 

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

vocab_size = tokenizer.vocab_size
embedding_dim = 128
hidden_dim = 256
num_layers = 2

class TokenTranslator(nn.Module):
    def __init__(self):
        super(TokenTranslator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.encoder(embedded)
        outputs, (hidden, cell) = self.decoder(outputs, (hidden, cell))
        predictions = self.fc_out(outputs)
        return predictions 


In [33]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TokenTranslator().to(device)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_epoch(model, data_loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in data_loader:
        src = batch['encrypted_input_ids'].to(device)
        trg = batch['input_ids'].to(device)

        optimizer.zero_grad()
        output = model(src)

        output = output.view(-1, vocab_size)
        trg = trg.view(-1)

        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)


In [35]:
def evaluate(model, data_loader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            src = batch['encrypted_input_ids'].to(device)
            trg = batch['input_ids'].to(device)

            output = model(src)
            output = output.view(-1, vocab_size)
            trg = trg.view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)


In [36]:
num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')


Epoch 1/5, Training Loss: 8.8651, Validation Loss: 6.3069
Epoch 2/5, Training Loss: 6.2781, Validation Loss: 5.8972
Epoch 3/5, Training Loss: 6.0321, Validation Loss: 5.7806
Epoch 4/5, Training Loss: 5.9504, Validation Loss: 5.7369
Epoch 5/5, Training Loss: 5.9154, Validation Loss: 5.7127
